# "2+1D" Pouch Cell Model

In this notebook we show how you can use PyBaMM to solve a "2+1D" pouch cell model. By "2+1D" we mean that the pouch cell is modelled by considering a number of 1+1D (pseudo-two-dimensional) models for the through-cell electrochemical behaviour coupled via a 2D model for the electrical behaviour in the current collectors. We also include thermal effects in the model, under the assumption that the temperature is uniform across the cell thickness (which is small compared with the remaining two dimensions). This results in a 2D model the thermal behaviour of cell. The model is fully-coupled, in the sense that the electrochimical behaviour (inclduding the parameters) depends on the temperature.

## Example solving a pouch cell model using PyBaMM

Below we show how to change the model options to solve a pouch cell model in PyBaMM.

First we need to import `pybamm` and the other packages we will use in this notebook, and then change our working directory to the root of the pybamm folder. 

In [2]:
import pybamm
import numpy as np
import os
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

We then need to choose our model and set the appropriate options. The flexibility of PyBaMM allows you to choose any battery model to describe the through cell behaviour. Here we choose the SPM, in the interest of simplicity. 

In order to turn the model into a pouch cell model we pass options telling PyBaMM which current collector model to use and what dimension the current current collector model should take. We choose the "potential pair" model and set the "dimensionality" to 2, since we want a "2+1D" model. 

Additionally, we set the thermal model to "x-lumped" to reflect our model assumption that the temperature doesn't vary in the through-cell direction $x$.

In [3]:
# load (2+1D) SPM model
options = {
    "current collector": "potential pair",
    "dimensionality": 2,
    "thermal": "x-lumped",
}
model = pybamm.lithium_ion.SPM(options)

We then use the default model geometry and parameters. Information about the cell geometry and tab location is stored in the parameter file.

In [4]:
# create geometry
geometry = model.default_geometry

# load parameter values and process model and geometry
param = model.default_parameter_values
param.process_model(model)
param.process_geometry(geometry)

We then set the mesh and discretise the model. In this notebook we use a relatively coarse mesh, just to illustrate how to set up the problem. You can experiment with the number of grid points. For large systems we recommend using a sparse solver (see the instructions for installing SUNDIALS with KLU sparse solver [here](https://github.com/pybamm-team/PyBaMM/blob/master/INSTALL-LINUX.md)).

In [5]:
# choose number of grid points per domain
var = pybamm.standard_spatial_vars
var_pts = {
    var.x_n: 5,
    var.x_s: 5,
    var.x_p: 5,
    var.r_n: 5,
    var.r_p: 5,
    var.y: 5,
    var.z: 5,
}
# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

We can now solve the model in the usual way

In [7]:
# solve model -- simulate one hour discharge
tau = param.evaluate(pybamm.standard_parameters_lithium_ion.tau_discharge)
t_end = 3600 / tau
t_eval = np.linspace(0, t_end, 120)
solution = pybamm.IDAKLUSolver().solve(model, t_eval)